#  QUESTION 1 Load the dataset into pandas and provide a summary of columns in the loaded dataset without any modification.


In [1]:
import pandas as pd #importing panda library 
import numpy as np #importing numpy library 
import seaborn as sns # importing seaborn library 
import matplotlib.pyplot as plt

In [ ]:
import os 
os.getcwd()

In [ ]:
df=pd.read_csv("Sales_April_2019.csv") #reading the data set
df #data set

In [ ]:
#So in this data set we have 6 coloums of sales of  electronics products.
#First Columns is of Order id and second column is of the product which is sold.
#Third columns tell the quantity of the product which is ordered .
#Fourth Column is of the price of the product.
#Fifth Column depicts about the order date and time.And Six Column is of the address of the purchaser.

In [ ]:
df1=df.copy() #doing deep copy to keep a copy of orginial data set
df1

# Data Information

In [ ]:
df.head(10) # Top 10 data of data set

In [ ]:
df.tail(10) # Last 10 data of data set

In [ ]:
df.info() #to check the data type and null value

# QUESTION 2 :-Prepare the data frame for further analysis. Clean it, mold it in any manner you see fit.

In [ ]:
# Data Cleaning
#Since there may be a chance of dubplicate data and null value
#So we have to find and drop it for further analaysis

In [ ]:
#since there are  373700 products and 372610 are non-null count products so there must be null product.Lets confirm in the next step.

In [ ]:
df.isna().sum() #to check the total null value

In [ ]:
#null value present in the data set so we have to drop it

In [ ]:
nan_df = df[df.isna().any(axis=1)] 
nan_df.head()

In [ ]:
df.dropna(how='all',inplace=True) #dropping nan value
df.isna().sum()              #checking if any nan value left

In [ ]:
#there is no Null values now.

In [ ]:
df.duplicated().value_counts() #checking dublicate value

In [ ]:
dfa=df.drop_duplicates(subset=None, keep='first', inplace=False) #droping the dubplicate values
dfa

In [ ]:
dfa.columns = ['Order_ID', 'Product', 'Quantity_Ordered', 'Price_Each', 'Order_Date', 'Purchase_Address']
dfa.head()

In [ ]:
dfa['Date'] = dfa['Order_Date'].str[:8]
dfa

In [ ]:
dfa['Month'] = dfa['Order_Date'].str[:2] #making a spearate column for month
dfa

In [ ]:
dfa[['House_No.','City','Pin_Code']] = dfa.Purchase_Address.str.split(",",expand=True)
dfa

In [ ]:
dfa.Month.value_counts() #to check the unique values #stucked here

In [ ]:
#Here we can see that "Or" value which is not a month so we are going to remove those rows which has Or values in Month Column

In [ ]:
dfa = dfa[dfa['Month'].str[:2] != 'Or']
dfa.head()

In [ ]:
dfa.shape

In [ ]:
dfa['Month'] = dfa['Month'].astype('int32')
dfa['Order_Date'] = pd.to_datetime(dfa['Order_Date'])
dfa['Date'] = pd.to_datetime(dfa['Date'])
dfa.info()

In [ ]:
dfa.Order_ID.value_counts()

In [ ]:
#orders are repeating so we cannot keep order id as index number

In [ ]:
dfa.Quantity_Ordered.value_counts()

In [ ]:
#All the values in Quantity Ordered column are Integer so have to convert it into Integer and Price Column to float.

In [ ]:
dfa['Quantity_Ordered'] = dfa['Quantity_Ordered'].astype('int32')
dfa['Price_Each'] = dfa['Price_Each'].astype('float64')
dfa.info()

In [ ]:
#now lets make the Sales column

In [ ]:
dfa['Sales'] = dfa['Quantity_Ordered'] * dfa['Price_Each']
dfa

# Summary

In [ ]:
# 1  In new data set we have separate coloumn for Time,date,Month,House_No., City, Pin_code
# 2  In new data set we can see the Sales Coloumn where it tells the price of total shopping
# 3  In data set all null values are eliminated
# 4  In data set order_id is repeating so we cannot keep it as index number
# 5  In old data set we had some null values,or in months all are cleaned in new data set.
# 5 Duplicate Data is removed from old data set

# Which city had the highest sales overall?

In [ ]:
city_sales=dfa.City.value_counts()
city_sales

In [ ]:
#From the value we can predict clearly that San Francisco giving maximum sale then Los Angeles
#lets plot the graph for clear view

In [ ]:
plt.figure(figsize=(16, 6))
sns.countplot(x=dfa['City'])
plt.xlabel("City Name")
plt.ylabel("Number of Sells")
plt.title("city vs Sales")

In [ ]:
#As we can see that San Fransisco has the highest sales followed by Los Angels.
#Austin City has lowest sales
#Seattle and Dallas has almost equal number of sales

# Question 4.c Most bought products per city?.

In [ ]:
result2=dfa.groupby('City').sum()
result2


In [ ]:
resultab = dfa.groupby('City')['Sales'].sum()
resultab

In [ ]:
cities=dfa['City'].unique()
cities

In [ ]:
cities=dfa['City'].unique()
cities = sorted(cities)
plt.bar(cities,result2['Sales'])
plt.xticks(cities, rotation='vertical',size =8)
label,location = plt.yticks()
plt.ylabel('Sales in Million USD')
plt.xlabel('City Name')
plt.show()

# Q5 Find out the distribution of sales over different times of the day (Morning, Afternoon, Evening, Night) and answer the following for each city

In [ ]:
dfa['Hour'] = dfa.Order_Date.dt.hour


In [ ]:
dfa.groupby('Hour').count()

In [ ]:
hours = [hour for hour, dfa in dfa.groupby('Hour')]
plt.plot(hours, dfa.groupby('Hour').count())
plt.grid(True)
plt.xticks(hours)
plt.xlabel('Hours in 24 Hr Format')
plt.ylabel('Orders')
plt.show()

In [ ]:
# 7pm most orders are placed because people like shopping after office or in the evening
# 3am -4 am is least  because most people must be sleeping
# 11am-12 noon is maximum after 7 pm 
# store should come with offer and discount in the night and early morning to encourage shopping 

# Q6 Find out the pair of products that are most sold together. For example, generally, we would expect a person who buys an iPhone 11 to also buy the Apple Airpods together with it. Identify such pairs of products and suggest what promotional offers should the retail company come up with based on your outcome?.
 

In [ ]:
order = dfa[dfa['Order_ID'].duplicated(keep = False)] 
order

In [ ]:
order['Ordered Together'] = order.groupby('Order_ID')['Product'].transform(lambda x : ','.join(x))
order = order[['Order_ID','Ordered Together']].drop_duplicates()
order.head(10)

In [ ]:
from itertools import combinations
from collections import Counter 

count = Counter() #the Counter object 'count' is used to count the occurrences of elements in the 'data_list'

for row in order['Ordered Together']:
    items = row.split(',')
    count.update(Counter(combinations(items,2))) # Most Common Pair of Products Bought Together.

for key, value in count.most_common(10): # Top 10 Products Pair Bought Together.
    print(key,value)

In [ ]:
# As we can see that iphone and light charging cabel is the most sold product as a pair.
# This can be because company doesnot provide charger while selling the phone
# second most sold product is usb c charging cabel and google phone


In [ ]:
product_group = dfa.groupby('Product')

quantity_ordered = product_group.sum()['Quantity_Ordered']

products = [product for product, detail in product_group]

In [ ]:
plt.figure(figsize = (20,5))
plt.bar(products, quantity_ordered)

plt.xticks(products, rotation = 'vertical')

plt.title('Most Sold Products')

plt.xlabel('Products')

plt.ylabel('Quantity of Products');

In [ ]:
# as we can see that aa and aaa batteries are most sold because we need it many uses.
# second the charging cabels are most sold because its malfunction more often so we need it regularly
# wired phone and apple airpods are also sold well
# so accessories are most sold 
#in mobile is vareebadd phone may be because its not good 
# lg company dryer and washing machine is least sale 
#so lg company is not doing well across all the city
#Offers:-
#lg company should come with offers to inrease there sales
#iphone should give chariging cables which may increases it sales
#lg can come with promotional offer by giving dryer free along with the washing machine.


# Conclusion

In [ ]:
# 1. data set are loaded
# 1a  It has  column of order id, quantity,order date addresss,price ,product
# 2  data is prepared for further analysis
# 2a Nan value present in data set are dropped
# 2b  Dublpicate values are deleted
# 2c  Column name are changed for better analysis
# 2d  Pincode ,city name are extracted and  given separate column
# 2e   separate sales column are made for further analysis
# 3 In new data set we donot have any nan values,dublicate values, we have separat column of sales,pincode,city.
# 3b  data types are changed of Order_ID for analysis
# 4q San Fransisco is the city where most sells are happening
# 4b New york city selling more products.
# 5 Most sales occuring at 7 evening may be because people like to shop in the evening.
 # 5.1 After 7pm its between 11am to 12 noon where most sales are happening.
 # 5.2 sales are least in 4 am morning which and decrease after 7pm so stores should offer some discount in night to increase sales at nigh

# 6 iphone and charging cables are most sold product together.
# 6.this is because iphone not giving charging cables along with mobile
# 6. lg items have least sales so company should offer discount.\
# 6. Accesories like headphone,charging cable, aa and aaa batteries are most sold item may be because of its  less durabiltiy.
# 6. There should be promotional offer like lg washing machine should offer lg dryer to increase its sales
# 6 veerband phone should offer discount or  free headphone and charging cabels.


 
